<a href="https://colab.research.google.com/github/christopher-ell/Deep_Learning_Begin/blob/master/5_Save_and_Restore_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://www.tensorflow.org/tutorials/keras/save_and_restore_models

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.12.0'

In [2]:
## Download data from the standard keras datasets dividing it into training and 
## testing data and data and labels
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

## Remove the first 10,000 labels
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

## Remove the first 10,000 images, reshape the matrix of images into a 
## vector and then divide by 255 so all cells are between 0 and 1
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


**Define Model**

In [3]:
# Returns a short sequential model
## Structure of model to be used
def create_model():
  model = tf.keras.models.Sequential([
      ## STack a layer with 512 nodes and a relu activation function.
      ## The number of parameters is equal to inputs (784) x nodes (512) + nodes (512)
      ## input_shape is input data shape
      keras.layers.Dense(512, activation = tf.nn.relu, input_shape = (784, )),
      ## Apply dropout as a form of regularisation to the cells of the activation 
      ## layer
      ## Drop out will randomly select 20% of the cells and make their value 0
      keras.layers.Dropout(0.2),
      ## The final layer will have 10 nodes and softmax activation function
      ## The number of parameters is equal to inputs (512) x 
      keras.layers.Dense(10, activation = tf.nn.softmax)
  ])
  
  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.sparse_categorical_crossentropy,
                metrics = ['accuracy'])

  return model

# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


**Saving Checkpoints During Training**

Checkpoint callback usage

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                save_weights_only = True,
                                                verbose = 1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
         validation_data = (test_images, test_labels),
         callbacks = [cp_callback]) # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.2132 - acc: 0.6507
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1636 - acc: 0.6670 - val_loss: 0.7159 - val_acc: 0.7790
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4207 - acc: 0.8889
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 329us/step - loss: 0.4201 - acc: 0.8850 - val_loss: 0.5294 - val_acc: 0.8190
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.3053 - acc: 0.9075
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 343us/step - loss: 0.3132 - acc: 0.9070 - val_loss: 0.4812 - val_

In [5]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 167us/step
Untrained model, accuracy:  6.20%


In [7]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 56us/step
Restored model, accuracy: 86.30%


Checkpoint callback Options

In [8]:
# Include the epoch in the file name. (uses 'str.format')
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
  checkpoint_path, verbose = 1, save_weights_only = True,
  # Save weights, every 5-epochs.
    period = 5)

model = create_model()
model.fit(train_images, train_labels,
         epochs = 50, callbacks = [cp_callback],
         validation_data = (test_images, test_labels),
         verbose = 0)
    


Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 000

In [9]:
! ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001  cp-0050.ckpt.index
cp-0025.ckpt.index


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 150us/step
Restored model, accuracy: 87.60%


**Manually Save Weights**

In [12]:
# Save the Weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the Weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 161us/step
Restored model, accuracy: 87.60%


**Save the Entire Model**

As a HDF5 File

In [13]:
model = create_model()

# You need to use a keras optimizer to restor the optimizer state from a HDF5 file.
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.sparse_categorical_crossentropy,
             metrics = ['accuracy'])

model.fit(train_images, train_labels, epochs = 5)

# Save entire model to HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 669us/step - loss: 1.1504 - acc: 0.6810
Epoch 2/5
1000/1000 [==============================] - 0s 254us/step - loss: 0.4193 - acc: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 256us/step - loss: 0.2833 - acc: 0.9230
Epoch 4/5
1000/1000 [==============================] - 0s 267us/step - loss: 0.2008 - acc: 0.9590
Epoch 5/5
1000/1000 [==============================] - 0s 258us/step - loss: 0.1510 - acc: 0.9710


In [14]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 211us/step
Restored model, accuracy: 85.70%


As Saved Model

In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs = 5)

Epoch 1/5
1000/1000 [==============================] - 1s 776us/step - loss: 1.1926 - acc: 0.6350
Epoch 2/5
1000/1000 [==============================] - 0s 275us/step - loss: 0.4168 - acc: 0.8910
Epoch 3/5
1000/1000 [==============================] - 0s 257us/step - loss: 0.2824 - acc: 0.9230
Epoch 4/5
1000/1000 [==============================] - 0s 255us/step - loss: 0.2079 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 260us/step - loss: 0.1654 - acc: 0.9600


In [17]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models/temp-b'1545186574'/saved_model.pb


In [18]:
! ls saved_models/

1545186574


In [19]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

In [20]:
# The optimizer was not restored
new_model.compile(optimizer = 'adam',
                 loss = tf.keras.losses.sparse_categorical_crossentropy,
                 metrics = ["accuracy"])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 471us/step
Restored model, accuracy: 85.70%
